In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sklearn
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
df_true = pd.read_csv("../input/fake-and-real-news-dataset/True.csv")
df_fake = pd.read_csv("../input/fake-and-real-news-dataset/Fake.csv")

In [ ]:
df_fake['label'] = 0
df_true['label'] = 1

In [ ]:
df = pd.concat([df_fake,df_true])

In [ ]:
df = df.drop('title', axis = 1)
df = df.drop('subject', axis = 1)
df = df.drop('date', axis = 1)

In [ ]:
df = sklearn.utils.shuffle(df)

In [ ]:
data = df['text']
labels = df['label']

In [ ]:
stop_words = list(stopwords.words('english')) 

In [ ]:
filterd_data = []
for words in data:
  if words not in stop_words:
    filterd_data.append(words)

In [ ]:
max_len = 500
max_features = 10000

In [ ]:
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(filterd_data)
sequence = tokenizer.texts_to_sequences(filterd_data)
ready_data = pad_sequences(sequence, maxlen = max_len)

In [ ]:
train_data = ready_data[:25000]
train_labels = labels[:25000]

validation_data = ready_data[25000:35000]
validation_labels = labels[25000:35000]

test_data = ready_data[35000:]
test_labels = labels[35000:]

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(train_data, train_labels, epochs = 10, validation_data= (validation_data, validation_labels))

In [ ]:
acc = history.history['acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label = 'acc')
plt.plot(epochs, val_acc , 'g', label = 'val_acc')
plt.title('acc')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label = 'loss')
plt.plot(epochs, val_loss , 'g', label = 'val_loss')
plt.title('loss')
plt.legend()
plt.figure()
plt.show()

In [ ]:
model.evaluate(test_data, test_labels)

In [ ]:
pred = model.predict_classes(test_data)
pred[:5]